# Elasticity workflow using MatMethods

In this excercise we will use the [MatMethods](https://github.com/hackingmaterials/MatMethods.git) package to compose VASP workflow to study the elastic properties of silicon.

In [ ]:
%matplotlib inline

import os
from helper_functions import *

db_file = os.path.abspath("db.json")

# Structure

In [ ]:
from pymatgen import Structure

struct_si = Structure([[5.468344455957462, 0.0, -0.0], 
                       [-0.0, 5.468344455957462, 0.0], 
                       [0.0, -0.0, 5.468344455957462]], 
                      
                      ["Si"]*8, 
                      
                      [[-0.0, -0.0, 0.5],
                       [0.75, 0.75, 0.75],
                       [0.0, 0.5, -0.0],
                       [0.75, 0.25, 0.25],
                       [0.5, 0.0, -0.0],
                       [0.25, 0.75, 0.25],
                       [0.5, 0.5, 0.5],
                       [0.25, 0.25, 0.75]])

# vasp inputset: MPRelaxSet

In [ ]:
from pymatgen.io.vasp.sets import MPRelaxSet, DictSet

vis = MPRelaxSet(struct_si, force_gamma=True)
vis.config_dict["KPOINTS"].update({"reciprocal_density": 600})
vis = DictSet(struct_si, vis.config_dict)

# Structure optimization firework

In [ ]:
from fireworks import Workflow
from matmethods.vasp.fireworks.core import OptimizeFW


vasp_cmd=None

fws = []
fw1 = OptimizeFW(struct_si, vasp_input_set=vis, vasp_cmd=vasp_cmd, db_file=db_file)
fws.append(fw1)

# Deformation fireworks

define normal and shear deformations

total 24 = 4 x 3 + 4 x 3

In [ ]:
from pymatgen.analysis.elasticity.strain import Deformation

norm_deformations=[-0.01, -0.005, 0.005, 0.01]
shear_deformations=[-0.06, -0.03, 0.03, 0.06]

deformations = []
#Normal deformations
for ind in [(0, 0), (1, 1), (2, 2)]:
    for amount in norm_deformations:
        defo = Deformation.from_index_amount(ind, amount)
        deformations.append(defo)
        
#Shear deformations
for ind in [(0, 1), (0, 2), (1, 2)]:
    for amount in shear_deformations:
        defo = Deformation.from_index_amount(ind, amount)
        deformations.append(defo)

Update incar and kpoint settings for the deformation fireworks

In [ ]:
#update incar file settings
def_incar_settings = vis.incar.as_dict()
def_incar_settings.update({"ISIF":2, "ISTART":1})

# remove unnecessary params
for key in ["MAGMOM", "@module", "@class", "LDAUU", "LDAUJ", "LDAUL"]:
    def_incar_settings.pop(key, None)
    
# deformation input param updatesL incar and kpoints
def_vasp_params = {"user_incar_settings":def_incar_settings, 
                  "reciprocal_density": 600}

Define firework for each deformation. A total of 24 deformation fireworks.

Uses the Transmuter firework to apply the deformation transformation

Also appends PassStressStrainData firetask to the Transmuter firework

In [ ]:
from fireworks import Firework
from matmethods.vasp.fireworks.core import TransmuterFW
from matmethods.vasp.workflows.base.elastic import PassStressStrainData

#generate deformation fireworks; one for each deformation
for i, deformation in enumerate(deformations):
    fw = TransmuterFW(name="elastic_deformation_"+str(i+1),
                      structure=struct_si,
                      transformations=['DeformStructureTransformation'],
                      transformation_params=[{"deformation": deformation.tolist()}],
                      copy_vasp_outputs=True,
                      db_file=db_file,
                      vasp_cmd=vasp_cmd,
                      parents=fws[0],
                      vasp_input_params=def_vasp_params
                     )
    fw.spec['_tasks'].append(PassStressStrainData(deformation=deformation.tolist()).to_dict())
    fws.append(fw)

# Workflow

In [ ]:
wfname = "{}:{}".format(struct_si.composition.reduced_formula, "elastic constants")
wf = Workflow(fws, name=wfname)

In [ ]:
plot_wf(wf, depth_factor=100.0, numerical_label=True, breadth_factor=12)

# Simulate vasp run

In [ ]:
wf = simulate_elasticity_vasprun(wf, deformations)

# Powerups

use the powerup to modify incar paramters: ENCUT and EDIFF

In [ ]:
from matmethods.vasp.vasp_powerups import add_modify_incar
from matmethods.vasp.workflows.presets.core import add_common_powerups

wf = add_modify_incar(wf, modify_incar_params={"incar_update": {"ENCUT": 700, 
                                                                "EDIFF": 1e-6}})

# Connect to Launchpad and reset

In [ ]:
from fireworks import LaunchPad

lp = LaunchPad()
lp.reset("", require_password=False)

# Submit Workflow

In [ ]:
lp.add_wf(wf)